In [ ]:
import sys
import os

spark_home='/opt/spark-1.5.1-bin-hadoop2.6/'
# addin pyspark to current path
sys.path.append( spark_home+'/python' )
sys.path.append( spark_home+'/python/lib/py4j-0.8.2.1-src.zip' )

# Установка локальных переменных
os.environ["SPARK_HOME"] = spark_home
os.environ["HADOOP_HOME"] = '/opt/cloudera/parcels/CDH/lib/hadoop'
os.environ["HADOOP_YARN_HOME"] = '/opt/cloudera/parcels/CDH/lib/hadoop-yarn'
os.environ["YARN_CONF_DIR"] = '/etc/hadoop/conf.cloudera.yarn'
os.environ["SPARK_CLASSPATH"] = '/etc/hive/conf.cloudera.hive1'
os.environ["PYSPARK_SUBMIT_ARGS"] = '--master local[4] pyspark-shell'

from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext
conf = SparkConf ().set( 'spark.app.name', 'test');
sc = SparkContext (conf= conf)

Наши датасеты для экспериментов
- examples - массив размеченных данных
- features - фичи массива examples

In [ ]:
from pyspark.mllib.util import MLUtils

examples = MLUtils.loadLibSVMFile(sc, "/testdata/mllib_demo/libsvm_dataset/")
features = examples.map(lambda x:x.features)
labels = examples.map(lambda x:x.label)
print features.first().size

In [ ]:
features.count()

# <br><br><br><br> Статистика фич

Вычисляется методом stat.colStats

In [ ]:
from pyspark.mllib.stat import Statistics
summary = Statistics.colStats(features)

Что есть полезного в результатах?
- среднее значение фичи summary.mean()
(всегод фич 119, рекомендуется взять первые 5)

In [ ]:
summary.numNonzeros()[:5]

- дисперсия

In [ ]:
summary.variance()

- минимум, максимум, число ненулевых элементов и т.д.

In [ ]:
summary.max()

# <br><br><br><br>Корреляция

Корреляция. Можно посчитать корреляцию как между разными элементами одного датасета,
так и двумя разными датасетами
Statistics.corr

Для примера найдем фичи, коррелирующие с целевой функцией

In [ ]:
features.map(lambda x: x[11]).collect()

In [ ]:
correlation = [(i, Statistics.corr(features.map(lambda x: x[i]), labels)) for i in xrange (features.first().size)]

Отсортировать по корреляции и взять топ 5 ответов (sorted ( ... reverse=True)

In [ ]:
sorted(correlation, reverse=True, key=lambda x:abs(x[1]))[:5]

Помимо корреляции двух векторов, corr позволяет вычислить корреляции всех столбцов в матрице между собой

In [ ]:
Statistics.corr (features)

# <br><br><br><br><br>Семплирование

Не входит в MLlib, но об этом стоит упомянуть при построении алгоритмов

**Случайные семплы**<br>
(для выделения обучающей и тестовой выборок)<br>
RDD.randomSplit([weight1, weight2,...], seed)

In [ ]:
(train, test) = examples.randomSplit([0.7, 0.3], 13)

In [ ]:
test.count()

**Стратифицированные семплы**<br>
Когда нужно отобрать заданную долю для каждого из значений.
Например в имеющемся датасете перекос по ключам<br>
fractions = {"a": 0.2, "b": 0.1}<br>
RDD.sampleByKey(withReplacement, fractions, seed)<br>
Работает только для датасетов вида ключ-значение

In [ ]:
# Выделяем целевую функцию в качестве ключа, считаем частоты
examples.map (lambda x: (x.label, x)).countByKey()

In [ ]:
examples.map (lambda x: (x.label, x))\
.sampleByKey (False,
              {0:0.33,  # 25% примеров с целевой функцией 0
               1:1},    # 100% примеров с целевой функцией 1
              42)\
.countByKey()

# <br><br><br><br><br><br><br><br><br>Генерация случайных данных

Есть распределения:
- **uniformRDD** (sc, size, numPartitions, seed)
- **normalRDD** (sc, size, numPartitions, seed)
- **poissonRDD** (sc, size, numPartitions, seed)


In [ ]:
from pyspark.mllib.random import RandomRDDs
# Пример нормального распределения m=0, sigma=1
RandomRDDs.normalRDD(sc, 1000, 10, 31).take(10)


# Переделываем распределение в m=100, sigma=0.01
RandomRDDs.uniformRDD(sc, 1000, 10, 31).take(10)

# <br><br><br><br><br>TF-IDF

Есть корпус документов $D$<br>
$t$ - некоторый термин в нем<br>
$d$ - некоторый документ в нем<br>
<br>
$TF(t,d)$ - сколько раз встретился термин $t$ в документе $d$<br>
<br>
$DF(t,D)$ - в скольки документах корпуса встечается термин $t$<br>
<br>
$IDF(t,D)=\log \frac{|D|+1}{DF(t,d)+1}$<br>
<br>
$TFIDF(t,d,D)=TF(t,d) \cdot IDF(t,D)$


В примере используется датасет из англоязычных абстрактов википедии<br>
"/testdata/mllib_demo/wiki_abstracts/abstrats1000.txt"

**TF**<br>
Частоты терминов считаются с использованием трюка с хешированием, чтобы не хранить связку термин-id<br>
hashingTF.transform(documents)

In [ ]:
from pyspark.mllib.feature import HashingTF

docs = sc.textFile (
"/testdata/mllib_demo/wiki_abstracts/abstrats1000.txt")
splitted_docs = docs.map (lambda x: x.split(" "))

hashingTF = HashingTF()
tf = hashingTF.transform(splitted_docs)

In [ ]:
tf.take(5)

**TF-IDF**<br>
Считается за 2 прохода
- сначала считается IDF<br> 
 IDF(minDocFreq).fit(tf)

In [ ]:
from pyspark.mllib.feature import IDF

idf = IDF(minDocFreq=2).fit(tf)

- затем умножается на TF<br>
 idf.transform(tf)

In [ ]:
tfidf = idf.transform(tf)

In [ ]:
tfidf.take(5)

##### <br><br><br><br><br> Пересчитаем TF-IDF из хешей в слова

Для начала надо построить словарь преобразования хеш - слово<br>
id2word = documents.flatMap (lambda x:x)\<br>
         .distinct ()\<br>
         .map (lambda x: (hashingTF.indexOf(x),x))\<br>
         .collectAsMap ()

Перебираем tf-idf и вместо каждого индекса подставляем слово
tfidf_words = tfidf.map (lambda doc:<br> 
                   zip(map (lambda x: (x,id2word [x]), doc.indices),<br>  doc.values))

In [ ]:
id2word = splitted_docs.flatMap (lambda x:x)\
.distinct ()\
.map (lambda x: (hashingTF.indexOf(x),x))\
.collectAsMap ()

In [ ]:
tfidf.map (lambda doc:
zip(map (lambda x: (x,id2word [x]), doc.indices),
doc.values)).take(5)

# <br><br><br><br><br>Масштабирование фич

Приведение фич к одному масштабу помогает сходимости итерационных методов поиска оптимумов

<table border="0">
<tr>
<td>
<img src='images/features_scaled_gd.png'>
</td>
<td>
<img src='images/features_normal_gd.png'>
</td>
</tr>
<tr>
<td align="center">
до масштабирования
</td>
<td align="center">
после масштабирования</td>
</tr>
</table>

In [ ]:
! hadoop fs -ls /testdata/mllib_demo/

In [ ]:
# Загружаем наш массив примеров
from pyspark.mllib.util import MLUtils
examples = MLUtils.loadLibSVMFile(sc, "/testdata/mllib_demo/libsvm_dataset")

# Выделяем фичи фичи примера
features = examples.map(lambda x:x.features)
labels = examples.map(lambda x:x.label)

Основной класс, который это делает - StandardScaler
- в начале строим модель преобразований<br> StandardScaler().fit(features)

In [ ]:
from pyspark.mllib.feature import StandardScaler

scaler1 = StandardScaler(withMean=True, withStd=True).fit(features)

In [ ]:
scaler1.transform (features.map(lambda x:x.toArray())).take(5)

- Затем преобразовываем по ней фичи<br>
model.transform(features)

**Внимание!**<br>
масштабирование с установкой среднего работает только с плотными векторами<br>
StandardScaler(withMean=True, withStd=True).fit(dense_features)

# <br><br><br><br><br><br><br>Нормализация семплов

При масштабировании мы работали с фичами. При нормализации - с семплами. 
Полезно для сравнения косинусного расстояния между двумя векторами
<img src='images/normalization.png'>

Запуск: Normalizer().transform(features)

In [ ]:
from pyspark.mllib.feature import Normalizer
Normalizer().transform(features).take(5)

<br><br><br><br>Можно нормализовывать фичи не по метрике $L^2$ а по $L^\infty$

Запуск: Normalizer(p=float("inf")).transform(features)

In [ ]:
Normalizer(p=float("inf")).transform(features).take (5)

## Word2vec

In [ ]:
from pyspark.mllib.feature import Word2Vec

In [ ]:
sentence

In [ ]:
sentence = "a b " * 100 + "a c " * 10
localDoc = [sentence, sentence]
doc = sc.parallelize(localDoc).map(lambda line: line.split(" "))
model = Word2Vec().setVectorSize(10).setSeed(42).fit(doc)

In [ ]:
model.findSynonyms("a",2)

In [ ]:
Word2Vec()